In [ ]:
import hopsworks
from datetime import datetime, timedelta
import synth_transactions as st  # if you keep module separation, else import functions directly

last_processed_date = datetime(2025, 9, 5)
current_date = datetime(2025, 10, 5)

project = hopsworks.login()
fs = project.get_feature_store()

name = "cc_trans_fg"

trans = fs.get_feature_group("credit_card_transactions")
cc_trans_aggs_fg = fs.get_feature_group("cc_trans_aggs_fg")
cc_fraud_fg = fs.get_feature_group("cc_fraud_fg")

cc_trans_fg = fs.get_or_create_feature_group(
    name=name,
    primary_key=["tid"],
    online_enabled=True,
    version=1,
    event_time="event_time",
    features=[        
        Feature("tid", type="bigint"),
        Feature("cc_num", type="string"),
        Feature("merchant_id", type="string"),
        Feature("amount", type="double"),
        Feature("ip_address", type="string"),
        Feature("card_present", type="double"),
        Feature("haversine_distance", type="bool"),
        Feature("time_since_last_trans", type="bigint"),
        Feature("days_to_card_expiry", type="bigint"),
        Feature("is_fraud", type="boolean"),
        Feature("event_time", type="timestamp"),
    ],    
)



In [ ]:
try:
    cc_trans_fg.save()
except Exception as e:
    print(e)

In [ ]:
trans_df = trans.read(start_time=last_processed_date)


In [ ]:
cc_trans_fg.insert(df)